# Breast Cancer Prediction 

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

---

## Background
For this illustration, we have taken an example for breast cancer prediction using UCI'S breast cancer diagnostic data set available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 

---

## Setup

Now we'll import the Python libraries we'll need.

In [7]:
!pip3 install tensorflow --user
!pip3 install autokeras --user
!pip3 install protobuf==3.8.0 --user

You are using pip version 19.0.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 1.2MB 19.4MB/s ta 0:00:01
coremltools 2.0 has requirement six==1.10.0, but you'll have six 1.13.0 which is incompatible.
You are using pip version 19.0.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json

import autokeras as ak

---
## Data

Data Source: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
        https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header = None)

# specify columns extracted from wbdc.names
data.columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean",
                "compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean",
                "radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se",
                "concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst",
                "perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst",
                "concave points_worst","symmetry_worst","fractal_dimension_worst"] 

# save the data
data.to_csv("data.csv", sep=',', index=False)

# print the shape of the data file
print(data.shape)

# show the top few rows
display(data.head())

# describe the data object
display(data.describe())

# we will also summarize the categorical field diganosis 
display(data.diagnosis.value_counts())


(569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


B    357
M    212
Name: diagnosis, dtype: int64

#### Key observations:
* Data has 569 observations and 32 columns.
* First field is 'id'.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features available for prediction.

## Create Features and Labels
#### Split the data into 80% training, 10% validation and 10% testing.

In [3]:
y = ((data.iloc[:,1] == 'M') +0).values;
X = data.iloc[:,2:].values;

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

---
## Train


In [8]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(max_trials=10) # It tries 10 different models.

clf.fit(X_train, y_train)


Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 6s - loss: 3.0857 - accuracy: 0.75 - 1s 66ms/step - loss: 1.7753 - accuracy: 0.5412 - val_loss: 0.7598 - val_accuracy: 0.7802
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.5436 - accuracy: 0.87 - 0s 5ms/step - loss: 0.4518 - accuracy: 0.8022 - val_loss: 0.4667 - val_accuracy: 0.8571
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.4266 - accuracy: 0.87 - 0s 5ms/step - loss: 0.3317 - accuracy: 0.8874 - val_loss: 0.2577 - val_accuracy: 0.9121
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.2729 - accuracy: 0.90 - 0s 5ms/step - loss: 0.2635 - accuracy: 0.9066 - val_loss: 0.2609 - val_accuracy: 0.9121
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.2746 - accuracy: 0.90 - 0s 5ms/step - loss: 0.2790 - accuracy: 0.8956 - val_loss: 0.2976 - val_accuracy: 0.8681
Epoch 6/1000
12/12 [==================

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 13s - loss: 0.9298 - accuracy: 0.406 - ETA: 0s - loss: 0.5834 - accuracy: 0.698 - 2s 127ms/step - loss: 0.5608 - accuracy: 0.7060 - val_loss: 2.9519 - val_accuracy: 0.4176
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.5161 - accuracy: 0.68 - 0s 7ms/step - loss: 0.3483 - accuracy: 0.8764 - val_loss: 2.4931 - val_accuracy: 0.4176
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.3634 - accuracy: 0.90 - 0s 8ms/step - loss: 0.3077 - accuracy: 0.8956 - val_loss: 1.7977 - val_accuracy: 0.4286
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.3271 - accuracy: 0.87 - 0s 8ms/step - loss: 0.2820 - accuracy: 0.9121 - val_loss: 1.3710 - val_accuracy: 0.4396
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.3551 - accuracy: 0.81 - 0s 8ms/step - loss: 0.2606 - accuracy: 0.9176 - val_loss: 1.1086 - val_accuracy:

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 6s - loss: 40.6628 - accuracy: 0.250 - 1s 72ms/step - loss: 28.5860 - accuracy: 0.4615 - val_loss: 8.0859 - val_accuracy: 0.7253
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 10.2916 - accuracy: 0.718 - 0s 6ms/step - loss: 14.5885 - accuracy: 0.6236 - val_loss: 2.1948 - val_accuracy: 0.8791
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 10.8588 - accuracy: 0.750 - 0s 5ms/step - loss: 10.1003 - accuracy: 0.7170 - val_loss: 2.4858 - val_accuracy: 0.8791
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 3.6739 - accuracy: 0.87 - 0s 5ms/step - loss: 7.2164 - accuracy: 0.8022 - val_loss: 1.8015 - val_accuracy: 0.8791
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 4.8978 - accuracy: 0.71 - 0s 5ms/step - loss: 6.5835 - accuracy: 0.8049 - val_loss: 2.5855 - val_accuracy: 0.8791
Epoch 6/1000
12/12 [=========

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 7s - loss: 3.3355 - accuracy: 0.59 - ETA: 0s - loss: 8.7795 - accuracy: 0.56 - 1s 83ms/step - loss: 7.4344 - accuracy: 0.5495 - val_loss: 1.3797 - val_accuracy: 0.7802
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 3.8638 - accuracy: 0.75 - ETA: 0s - loss: 3.7476 - accuracy: 0.62 - 0s 10ms/step - loss: 3.1374 - accuracy: 0.6538 - val_loss: 0.4678 - val_accuracy: 0.8352
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.9500 - accuracy: 0.68 - ETA: 0s - loss: 1.8744 - accuracy: 0.65 - 0s 10ms/step - loss: 1.6900 - accuracy: 0.6841 - val_loss: 0.3045 - val_accuracy: 0.8681
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.2455 - accuracy: 0.90 - ETA: 0s - loss: 0.6370 - accuracy: 0.76 - 0s 10ms/step - loss: 0.6280 - accuracy: 0.7802 - val_loss: 0.2358 - val_accuracy: 0.9231
Epoch 5/1000
12/12 [==============================]

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 13s - loss: 0.6127 - accuracy: 0.656 - 2s 126ms/step - loss: 0.3869 - accuracy: 0.8626 - val_loss: 7.9654 - val_accuracy: 0.4176
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.3719 - accuracy: 0.78 - 0s 7ms/step - loss: 0.2860 - accuracy: 0.9011 - val_loss: 5.2018 - val_accuracy: 0.4176
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.2996 - accuracy: 0.90 - 0s 8ms/step - loss: 0.2620 - accuracy: 0.9176 - val_loss: 3.2793 - val_accuracy: 0.4286
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.3174 - accuracy: 0.84 - 0s 8ms/step - loss: 0.2437 - accuracy: 0.8984 - val_loss: 2.0846 - val_accuracy: 0.4396
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.2902 - accuracy: 0.87 - 0s 8ms/step - loss: 0.2154 - accuracy: 0.9286 - val_loss: 1.1727 - val_accuracy: 0.4945
Epoch 6/1000
12/12 [===============

...

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 6s - loss: 29.4807 - accuracy: 0.250 - 1s 67ms/step - loss: 3.8431 - accuracy: 0.7115 - val_loss: 0.3426 - val_accuracy: 0.8901
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.2774 - accuracy: 0.90 - 0s 6ms/step - loss: 1.0663 - accuracy: 0.8022 - val_loss: 0.4100 - val_accuracy: 0.9121
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.4233 - accuracy: 0.84 - 0s 6ms/step - loss: 0.5181 - accuracy: 0.9038 - val_loss: 0.3105 - val_accuracy: 0.9341
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.2819 - accuracy: 0.87 - 0s 5ms/step - loss: 0.8888 - accuracy: 0.8324 - val_loss: 0.2910 - val_accuracy: 0.9341
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.2227 - accuracy: 0.90 - 0s 5ms/step - loss: 0.3950 - accuracy: 0.9258 - val_loss: 0.2859 - val_accuracy: 0.9341
Epoch 6/1000
12/12 [================

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 6s - loss: 3.7648 - accuracy: 0.12 - 1s 68ms/step - loss: 1.6340 - accuracy: 0.6071 - val_loss: 0.6116 - val_accuracy: 0.8462
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.3150 - accuracy: 0.81 - 0s 5ms/step - loss: 0.4210 - accuracy: 0.8791 - val_loss: 0.4584 - val_accuracy: 0.8791
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.3935 - accuracy: 0.81 - 0s 5ms/step - loss: 0.4026 - accuracy: 0.8929 - val_loss: 0.3904 - val_accuracy: 0.8681
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.3612 - accuracy: 0.87 - 0s 4ms/step - loss: 0.3725 - accuracy: 0.9011 - val_loss: 0.3392 - val_accuracy: 0.8681
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.3317 - accuracy: 0.87 - 0s 5ms/step - loss: 0.3465 - accuracy: 0.9011 - val_loss: 0.2952 - val_accuracy: 0.8791
Epoch 6/1000
12/12 [==================

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 5s - loss: 16.5210 - accuracy: 0.250 - 1s 63ms/step - loss: 2.7955 - accuracy: 0.6401 - val_loss: 0.4843 - val_accuracy: 0.8352
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.5955 - accuracy: 0.84 - 0s 5ms/step - loss: 0.4059 - accuracy: 0.9066 - val_loss: 0.2572 - val_accuracy: 0.9231
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.3187 - accuracy: 0.87 - 0s 5ms/step - loss: 0.3800 - accuracy: 0.9011 - val_loss: 0.2514 - val_accuracy: 0.9341
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.2845 - accuracy: 0.90 - 0s 5ms/step - loss: 0.3970 - accuracy: 0.8736 - val_loss: 0.4053 - val_accuracy: 0.8681
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.4306 - accuracy: 0.84 - 0s 5ms/step - loss: 0.3363 - accuracy: 0.9066 - val_loss: 0.2572 - val_accuracy: 0.9121
Epoch 6/1000
12/12 [================

Train for 12 steps, validate for 3 steps
Epoch 1/1000
12/12 [==============================] - ETA: 13s - loss: 0.7827 - accuracy: 0.468 - 2s 125ms/step - loss: 0.6753 - accuracy: 0.6126 - val_loss: 1.5329 - val_accuracy: 0.4176
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.5479 - accuracy: 0.81 - 0s 5ms/step - loss: 0.4053 - accuracy: 0.8571 - val_loss: 1.8094 - val_accuracy: 0.4176
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.3806 - accuracy: 0.90 - 0s 5ms/step - loss: 0.3146 - accuracy: 0.9011 - val_loss: 1.6107 - val_accuracy: 0.4176
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 0.3594 - accuracy: 0.90 - 0s 7ms/step - loss: 0.2875 - accuracy: 0.9176 - val_loss: 1.2718 - val_accuracy: 0.4505
Epoch 5/1000
12/12 [==============================] - ETA: 0s - loss: 0.3070 - accuracy: 0.90 - 0s 6ms/step - loss: 0.2589 - accuracy: 0.9258 - val_loss: 0.9504 - val_accuracy: 0.5714
Epoch 6/1000
12/12 [===============

INFO:tensorflow:Oracle triggered exit
Train for 15 steps, validate for 3 steps
Epoch 1/1000
15/15 [==============================] - ETA: 16s - loss: 0.7412 - accuracy: 0.500 - ETA: 0s - loss: 0.4707 - accuracy: 0.785 - 2s 102ms/step - loss: 0.4435 - accuracy: 0.8088 - val_loss: 5.4618 - val_accuracy: 0.4176
Epoch 2/1000
15/15 [==============================] - ETA: 0s - loss: 0.4131 - accuracy: 0.84 - ETA: 0s - loss: 0.3161 - accuracy: 0.89 - 0s 7ms/step - loss: 0.3076 - accuracy: 0.9011 - val_loss: 3.0701 - val_accuracy: 0.4176
Epoch 3/1000
15/15 [==============================] - ETA: 0s - loss: 0.3281 - accuracy: 0.90 - ETA: 0s - loss: 0.2679 - accuracy: 0.92 - 0s 7ms/step - loss: 0.2596 - accuracy: 0.9297 - val_loss: 1.7006 - val_accuracy: 0.4396
Epoch 4/1000
15/15 [==============================] - ETA: 0s - loss: 0.3069 - accuracy: 0.87 - ETA: 0s - loss: 0.2267 - accuracy: 0.91 - 0s 7ms/step - loss: 0.2205 - accuracy: 0.9231 - val_loss: 1.0855 - val_accuracy: 0.5604
Epoch 5/1000

In [9]:
print(clf.evaluate(X_val, y_val))

4/4 [==============================] - ETA: 0s - loss: 0.3149 - accuracy: 0.90 - 0s 63ms/step - loss: 0.2213 - accuracy: 0.9386
[0.22128284350037575, 0.9385965]


In [10]:
# Export as a Keras Model.
model = clf.export_model()

In [13]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
categorical_encoding (Catego (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1024)              31744     
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                32800 